In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
# from wordcloud import WordCloud

In [ ]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader, SlackDataProcessor
from src.utils import Utils

In [ ]:
slack_parser = SlackDataProcessor().slack_parser
parse_slack_reaction = SlackDataProcessor().parse_slack_reaction
get_messages_dict = Utils().get_messages_dict

In [ ]:
slack_folder_path = os.path.abspath('../anonymized')

# Create an instance of SlackDataLoader
slack_data_loader = SlackDataLoader(slack_folder_path)

# slack_users_data and slack_channels_data Slack data
slack_users_data = slack_data_loader.get_users()
slack_channels_data = slack_data_loader.get_channels()

channel_names = [channel['name'] for channel in slack_channels_data]
user_names = [user['name'] for user in slack_users_data]

In [ ]:
parsed_data = slack_parser('../anonymized/all-community-building/')

In [ ]:
def slack_dataframe(channel_names, root_dir='../anonymized/'):
    slack_dataframe = []

    for channel_name in channel_names:
        channel_path = os.path.join(root_dir, channel_name)
        channel_dataframe = slack_parser(channel_path)
        slack_dataframe.append(channel_dataframe)

    merged_data = pd.concat(slack_dataframe, ignore_index=True)

    return merged_data
merged_data = slack_dataframe(channel_names)

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [ ]:
def get_most_active_users(df, top_n=10, ascending=False):
    # Get value counts of 'sender_name' column
    user_reply_counts = df['sender_name'].value_counts()

    # Get the top or bottom N users by reply count
    top_users = user_reply_counts.head(top_n) if not ascending else user_reply_counts.tail(top_n)

    return top_users

def get_least_active_users(df, bottom_n=10):
    # Get value counts of 'sender_name' column
    user_reply_counts = df['sender_name'].value_counts()

    # Get the bottom N users by reply count
    bottom_users = user_reply_counts.tail(bottom_n)

    return bottom_users

In [ ]:
get_most_active_users(merged_data)
get_least_active_users(merged_data)

In [ ]:
def get_most_active_users_by_message_count(df, top_n=10, ascending=False):
    # Group the DataFrame by 'sender_name' and calculate the count of unique messages for each user
    user_message_num = df.groupby('sender_name')['msg_content'].nunique()

    # Sort the user_message_num Series based on the specified order
    sorted_user_message_num = user_message_num.sort_values(ascending=ascending)

    # Get the top or bottom users by message count
    top_users = sorted_user_message_num.head(top_n)

    return top_users

def get_least_active_users_by_message_count(df, bottom_n=10):
    # Group the DataFrame by 'sender_name' and calculate the count of unique messages for each user
    user_message_num = df.groupby('sender_name')['msg_content'].nunique()

    # Sort the user_message_num Series in ascending order
    sorted_user_message_num = user_message_num.sort_values(ascending=True)

    # Get the bottom users by message count
    bottom_users = sorted_user_message_num.head(bottom_n)

    return bottom_users

In [ ]:
get_least_active_users_by_message_count(merged_data)
get_most_active_users_by_message_count(merged_data)

In [ ]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:
# which user has the highest number of reply counts?

In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [11]:
# what kind of messages are replied faster than others?

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?